# Seasonal Adjustment

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import datetime as dt
import seaborn as sns
from sklearn import base, linear_model
import dill as pickle
mpl.rcParams['savefig.dpi'] = 1.5 * mpl.rcParams['savefig.dpi']

#### adjust_date
This function takes a pandas DataFrame and artifically adjusts the date column.
The default column name is 'date'. The type of date adjustment is defaulted to
no change, the user must select year, month or week.

    year: set all years to 2000 (a leap year)
    month: set all days to 01 (overrides week)
    week: set days to either 01, 08, 15 or 22
    
The date is expected as a string in the form YYYY-MM-DD
The function returns the input DataFrame but with the date column adjusted
and stored as a datetime object.

In [ ]:
def adjust_date(df, date_col='date', year=False, month=False, week=False):
    if year: df[date_col] = df[date_col].map(lambda x: x.replace(x[:4], '2000'))
    if month: df[date_col] = df[date_col].map(lambda x: x.replace(x[8:], '01'))
    if week:
        i = 0
        for d in df[date_col]:
            if int(d[8:]) < 8: new_day = '01'
            elif int(d[8:]) < 15: new_day = '08'
            elif int(d[8:]) < 22: new_day = '15'
            else: new_day = '22'
            df.at[i,date_col] = d.replace(d[8:], new_day)
            i = i + 1
    # Convert 'date' column from strings to a datetimes
    df[date_col] = pd.to_datetime(df[date_col], format='%Y-%m-%d')
    return df;

In [ ]:
business_datafile = '~/capstone/data/yelp_academic_dataset_business.csv'
biz_id = 16  # Column containing the business_id, variable used as dataframe index name
user_datafile = '~/capstone/data/yelp_academic_dataset_user.csv'
usr_id = 16  # Column containing the user_id, variable used as dataframe index name
review_datafile = '~/capstone/data/yelp_academic_dataset_review.csv'
rev_id = 1   # Column containing the review_id, variable used as dataframe index name

business = pd.read_csv(business_datafile, index_col=biz_id)
user = pd.read_csv(user_datafile, index_col=usr_id)
review = pd.read_csv(review_datafile, index_col=rev_id)

Trim data to just consider restaurants and convert date column to datetime objects

In [ ]:
rest_crit = business['categories'].map(lambda x: 'Restaurants' in x)
restaurants = business[rest_crit]
restaurant_ids = restaurants.index.values
rest_reviews = review[review['business_id'].isin(restaurant_ids)]

rest_reviews = adjust_date(rest_reviews)

In [ ]:
print restaurants.shape

Set the minimum number of reviews to consider to build the model. Get new DataFrames that have businesses greater than the minimum number of reviews and use that list of restaurants to cull the review DataFrame.

In [ ]:
min_reviews = 20

rest_train = restaurants[restaurants.review_count >= min_reviews]
rest_ids = rest_train.index.values
reviews_train = rest_reviews[rest_reviews['business_id'].isin(rest_ids)]
reviews_train.drop(['votes.cool','votes.funny','votes.useful','type'], axis=1, inplace=True)
reviews_train.head()

Let's build a model to describe the seasonal variation of reviews. We need to be careful to account for the fact that the number of reviews for a given restaurant are not evenly distributed throughout the year. There maybe more (or less) reviews during times where the reviews will be higher than expected. Building this model requires:

1. Finding the mean star rating of each restaurant *independent* of the number of reviews. This is accomplished through grouping the review DataFrame by both business_id and date and taking the mean.
2. Given a DataFrame grouped by business_id and date, we need to group again by business_id to get a final mean rating for a restaurant. This final mean rating should be a closer approximation to the time independent rating of the restaurant. However, if the number of reviews are small or happen during high/low times, it might be off.
3. Next, subtract this new mean rating (from the previous step) from the second grouped by DataFrame (from the previous step) to create a new DataFrame of mean adjusted ratings.


In [ ]:
grouped_reviews = reviews_train.groupby(['business_id','date'], as_index=False).mean()
grouped_reviews.tail()

In [ ]:
avg_per_rest = grouped_reviews.groupby('business_id', as_index=False).mean()
avg_per_rest.rename(columns = {'stars':'avg_stars'}, inplace = True)
avg_per_rest.head()

Not really necessary, but good to know

In [ ]:
avg_all_rest = avg_per_rest["avg_stars"].mean()
print 'Average star rating of all the restaurants: {}'.format(avg_all_rest)

If our assumption that seasonal adjustments affect the ratings is correct, then we expect to see similiar adjustements per metro area. We need to add the metro area to reviews DataFrame. As a proxy for metro area, we will use state. We must make allowances for those metro areas which have multiple states.

In [ ]:
tmp = restaurants.reset_index()
state = tmp[['business_id','state']]
print state.state.unique()
# Combine multi-state metro areas to a single state metro-area
# Drop the clearly mislabeled/bad data
# XGL - is the code for greater London
# NW - is the code for Nordrhein-Westfalen, not near Karlsrhue
state.state.replace(to_replace=['SC','MLN','FIF','ELN','BW','RP'],
                    value=['NC','EDH','EDH','EDH','KHL','KHL'],
                    inplace=True)
state = state[state.state != 'XGL']
state = state[state.state != 'NW']
print state.state.unique()

In [ ]:
grouped_reviews = pd.merge(grouped_reviews, avg_per_rest, on='business_id')
grouped_reviews['stars_adj'] = grouped_reviews.stars - grouped_reviews.avg_stars
grouped_reviews = pd.merge(grouped_reviews, state, on='business_id')
grouped_reviews.tail()

### Arizona

Added cutoff date to make sure that we have a review on every day. I am doing this so I can run an FFT without worrying about missing data

In [ ]:
# AZ = grouped_reviews[grouped_reviews['state'] == 'AZ'].groupby('date', as_index=False).agg(['mean', 'std', 'count'])
cutoff_date = dt.date(2008, 1, 1)
AZ = grouped_reviews[(grouped_reviews.state == 'AZ') & (grouped_reviews.date > cutoff_date)]
AZ.head()

In [ ]:
AZ_gb_date = AZ.groupby('date', as_index=False).mean()
AZ_gb_date.tail()

In [ ]:
plt.plot(AZ_gb_date.date, AZ_gb_date.stars_adj, 'b.', label='data', alpha=0.5)
plt.xlabel('Date')
plt.ylabel('Star Rating')
plt.legend(loc='upper right')
plt.gcf().autofmt_xdate()
plt.show()

In [ ]:
y = np.abs(np.fft.rfft(AZ_gb_date.stars_adj))

n = AZ_gb_date.stars_adj.size
timestep = 1.
x = np.fft.rfftfreq(n, d=timestep)
plt.title("PSD of ratings AZ adjusted stars")
plt.xlabel("Frequency [cycles/day]")
plt.ylabel("Power")
plt.plot(x, y**2)
plt.show()

xran = [.1,.2]
#nlabels = 8
#xlabels = np.arange(nlabels)*(xran[1]-xran[0])+xran[0]
plt.plot(x, y**2, color='dodgerblue', alpha=0.5)
plt.ylim([0,800])
#plt.xlim([150.,1450.])
plt.xlim(xran)
#plt.xticks(range(nlabels),xlabels)
plt.title("PSD of Arizona Restaurants")
plt.ylabel("Power")
plt.xlabel("Frequency [cycles/day]")
plt.show()

plt.plot(x, y**2)
plt.ylim([0,800])
plt.xlim([0,.04])
plt.show()

In [ ]:
f1 = np.argmax(y[1:1000])+1
print f1, x[f1]
print 'peak: {} cycles/week'.format(x[f1]*7.)
f2 = np.argmax(y[:200])
print f2, x[f2]
print 'peak: {} cycles/year'.format(x[f2]*365.25)
f3 = np.argmax(y[f2+1:200])+f2+1
print f3, x[f3]
print 'peak: {} cycles/year'.format(x[f3]*365.25)

### Pittsburgh

In [ ]:
cutoff_date = dt.date(2008, 1, 1)
PA = grouped_reviews[(grouped_reviews.state == 'PA') & (grouped_reviews.date > cutoff_date)]
PA.head()

In [ ]:
PA_gb_date = PA.groupby('date', as_index=False).mean()
PA_gb_date.tail()

In [ ]:
plt.plot(PA_gb_date.date, PA_gb_date.stars_adj, 'b.', label='data')
plt.xlabel('Date')
plt.ylabel('Star Rating')
plt.legend(loc='upper right')
plt.gcf().autofmt_xdate()
plt.show()

In [ ]:
y = np.abs(np.fft.rfft(PA_gb_date.stars_adj))

n = PA_gb_date.stars_adj.size
timestep = 1.
x = np.fft.rfftfreq(n, d=timestep)

plt.title("PSD of ratings PA adjusted stars")
plt.plot(x, y**2)
plt.show()

plt.plot(x, y**2)
plt.ylim([0,4000])
plt.xlim([.1,.2])
plt.show()

plt.plot(x, y**2)
plt.ylim([0,4000])
plt.xlim([0,.04])
plt.show()

In [ ]:
f1 = np.argmax(y[1:4000])+1
print f1, x[f1]
print 'peak: {} cycles/week'.format(x[f1]*7.)

In [ ]:
top_freq = [x[i] for i,v in enumerate(y[:100]**2) if v > 2100]
for f in top_freq:
    print 'peak: {} cycles/year'.format(f*365.25)

# Las Vegas

In [ ]:
cutoff_date = dt.date(2008, 1, 1)
NV = grouped_reviews[(grouped_reviews.state == 'NV') & (grouped_reviews.date > cutoff_date)]
NV.head()
NV_gb_date = NV.groupby('date', as_index=False).mean()
NV_gb_date.tail()

In [ ]:
plt.plot(NV_gb_date.date, NV_gb_date.stars_adj, 'b.', label='data')
plt.xlabel('Date')
plt.ylabel('Star Rating')
plt.legend(loc='upper right')
plt.gcf().autofmt_xdate()
plt.show()

In [ ]:
y = np.abs(np.fft.rfft(NV_gb_date.stars_adj))

n = NV_gb_date.stars_adj.size
timestep = 1.
x = np.fft.rfftfreq(n, d=timestep)

plt.title("PSD of ratings NV adjusted stars")
plt.plot(x, y**2)
plt.ylim([0,500])
plt.show()

plt.plot(x, y**2)
plt.ylim([0,500])
plt.xlim([.1,.2])
plt.show()

plt.plot(x, y**2)
plt.ylim([0,500])
plt.xlim([0,.04])
plt.show()

In [ ]:
f1 = np.argmax(y[100:1000])+100
print f1, x[f1]
print 'peak: {} cycles/week'.format(x[f1]*7.)

In [ ]:
top_freq = [x[i] for i,v in enumerate(y[:1000]**2) if v > 300]
for f in top_freq:
    print 'peak: {} cycles/year'.format(f*365.25)

### What's the take away message?

It is clear there is some cyclical variation on both annual and weekly cycles. There maybe more funky issues that come up on an annual (monthly?) cycle that is not a simple sine/cosine with a period of one year. (Think a step function for a few months) Pressing ahead, I am going to go back to all the data and create new features of the sine/cosine by the year and week. Next, I will fit those new features to a per city model and store the model. This model will give me the expected change in star rating given the date. I can then apply this model to all of my restaurants and get an adjustment for every review. Get the average of that to come up with an average correction based on the date.

The next step of the model would be to investigate if the adjustments were related to sun or temperature or something else instead of a general day of the year thing. I would have to get sun and temperature data for each city to build a better(?) model.

In [ ]:
# Add columns to DataFrame for new features
new_feat = grouped_reviews.copy()
new_feat['day'] = grouped_reviews.date.apply(lambda x: x.dayofyear)
new_feat['day_of_week'] = grouped_reviews.date.apply(lambda x: x.dayofweek)
new_feat['sin(day)'] = np.sin(new_feat['day'] / 365. * 2. * np.pi)
new_feat['cos(day)'] = np.cos(new_feat['day'] / 365. * 2. * np.pi)
new_feat['sin(week)'] = np.sin(new_feat['day_of_week'] / 7. * 2. * np.pi)
new_feat['cos(week)'] = np.cos(new_feat['day_of_week'] / 7. * 2. * np.pi)
new_feat.head()

In [ ]:
class DateModel(base.BaseEstimator, base.RegressorMixin):
    def __init__(self, estimator):
        # initialization code
        self.est = estimator
        self.func = {}
        return
        
    def fit(self, X, Y=None):
        # fit the model ..-
        for state, df in X.groupby(['state'], sort=False):
            linreg = self.est()
            my_est = linreg.fit(df[['sin(day)', 'cos(day)', 'sin(week)', 'cos(week)']], df['stars_adj'])
            self.func[state] = my_est
        return self

    def predict(self, X):
        # prediction
        key = X['state']
        if key in self.func.keys():
            if isinstance(X,dict):
                return self.func.get(key).predict([X['sin(day)'], X['cos(day)'],
                                                   X['sin(week)'], X['cos(week)']])
            else:
                return self.func.get(key).predict(X[['sin(day)', 'cos(day)', 'sin(week)', 'cos(week)']])
        else:
            return -9999

In [ ]:
my_date_model = DateModel(linear_model.Ridge)
my_date_model.fit(new_feat)

In [ ]:
N = 3000
X = new_feat.iloc[N]
Y = X['stars_adj']

print 'Prediction = {}, Actual rating = {}'.format(my_date_model.predict(X)[0],Y)

In [ ]:
filename = '/home/vagrant/capstone/data/date_model.pkl'
with open(filename,'w') as f:
    pickle.dump(my_date_model, f)

In [ ]:
rest_reviews = review[review['business_id'].isin(restaurant_ids)].copy()
rest_reviews = adjust_date(rest_reviews)
rest_reviews.drop(['votes.cool','votes.funny','votes.useful','type'], axis=1, inplace=True)
rest_reviews_state_corrected = pd.merge(rest_reviews, state, on='business_id')

# Add columns to DataFrame for new features
rest_views = rest_reviews_state_corrected.copy()
rest_views['day'] = rest_reviews_state_corrected.date.apply(lambda x: x.dayofyear)
rest_views['day_of_week'] = rest_reviews_state_corrected.date.apply(lambda x: x.dayofweek)
rest_views['sin(day)'] = np.sin(rest_views['day'] / 365. * 2. * np.pi)
rest_views['cos(day)'] = np.cos(rest_views['day'] / 365. * 2. * np.pi)
rest_views['sin(week)'] = np.sin(rest_views['day_of_week'] / 7. * 2. * np.pi)
rest_views['cos(week)'] = np.cos(rest_views['day_of_week'] / 7. * 2. * np.pi)
rest_views.head()

In [ ]:
def dumb(x):
    return my_date_model.predict(x)[0]

In [ ]:
model_corrected = rest_views.apply(lambda x: my_date_model.predict(x)[0], axis = 1)

In [ ]:
model_corrected_reviews = rest_views.copy()
model_corrected_reviews['date_correction'] = model_corrected
model_corrected_reviews.tail()

In [ ]:
filename = '/home/vagrant/capstone/data/model_corrected_reviews.pkl'
with open(filename,'w') as f:
    pickle.dump(model_corrected_reviews, f)

In [ ]:
filename = '/home/vagrant/capstone/data/model_corrected_reviews.pkl'
with open(filename,'r') as f:
    model_corrected_reviews = pickle.load(f)

In [ ]:
mean_mc_reviews = model_corrected_reviews.groupby('business_id', as_index=False).mean()
mean_mc_reviews.drop(['day','day_of_week','sin(day)','cos(day)','sin(week)','cos(week)'], axis=1, inplace=True)
mean_mc_reviews['date_corrected_stars'] = mean_mc_reviews.stars - mean_mc_reviews.date_correction
mean_mc_reviews.head()

In [ ]:
print 'The maximum restaurant OVER-rating is {}'.format(mean_mc_reviews.date_correction.max())
print 'The maximum restaurant UNDER-rating is {}'.format(mean_mc_reviews.date_correction.min())

In [ ]:
filename = '/home/vagrant/capstone/data/date_adjusted_ratings.pkl'
with open(filename,'w') as f:
    pickle.dump(mean_mc_reviews, f)